# LIBS

In [ ]:
from sklearn.linear_model import SGDClassifier
## Scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay

## Librerias para graficación
import matplotlib.pyplot as plt
import seaborn as sns

# NLTK es una librería particular para PLN. Tiene muchas funcionalidades entre ellas stemming y lista de palabras de parada.
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd

stemmer = nltk.stem.SnowballStemmer('english') # Vamos a utlizar el Snowball Stemmer para realizar stemming (nos permite llevar las palabras a una forma estandar).
nltk.download('stopwords')


# DATASET

In [ ]:
df = pd.read_csv('train.csv', sep=',', header=0, index_col= None, engine='python',
                 usecols=['text','emotion'])

# CLASSIFIERS

In [ ]:
'''
* processing_text
* @param texto str
* @return processed_feature str
'''
def processing_text(texto):
    # Paso 1: Remover con un expresión regular carateres especiales (no palabras).
    processed_feature = re.sub(r'\W', ' ', str(texto))
    # Paso 2: Remover ocurrencias de caracteres individuales
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature)
    # Paso 3: Remover números (Ocurrencias muy esporádicas en nuestro dataset)
    processed_feature = re.sub(r'[0-9]+', ' ', processed_feature)
    # Paso 4: Simplificar espacios concecutivos a un único espacio entre palabras
    processed_feature = re.sub(' +', ' ', processed_feature)
    # Paso 5: Pasar todo el texto a minúsculas
    processed_feature = processed_feature.lower()
    # Paso 6: Aplicar stemming. Es una forma de enviar las palabras a una raiz común simplificando de esta manera el vocabulario.
    #         por ejemplo las palabras (absurdo, absurdos) que estan en el review 2895 seran llevados a la raiz común "absurd"
    #         y de esta forma se evita tener dos palabras diferentes con el mismo significado en nuestro vocabulario.
    processed_feature = " ".join([stemmer.stem(i) for i in processed_feature.split()])


    return processed_feature

Procesamiento de texto: No contemple números, pase a minúscula, elimine caracteres individuales,
tokenize por palabra, y aplique stemming.


In [ ]:
# Primero vamos extraer del dataframe la columna texto y la polaridad y las almacenaremos en las variables
# texto_para_procesar y labels respectivamente
texto_para_procesar = (df['text'].values)
labels = df['emotion'].values

# El texto ya procesado de cada ejemplo en nuestro dataset lo almacenaremos en la variable "texto_procesado"
texto_procesado = []
for sentence in range(0, len(texto_para_procesar)):
    procesado = processing_text(texto_para_procesar[sentence])
    texto_procesado.append(procesado)

Funcion del modelo

In [ ]:
def create_model(texto_features, model, eta=0.1):
    # Partición del dataset: Seleccionar 80% para entrenamiento, 20% pruebas.
    X_train, X_test, y_train, y_test = train_test_split(
        texto_features, labels, test_size=0.2, random_state=0)

    # Modelo: Naive Bayes | SDGClassifier
    if model == "naives":
        model = MultinomialNB()
        model.fit(X_train, y_train)
    else:
        model = SGDClassifier(
            loss='log_loss', learning_rate='constant', eta0=eta)
        model.fit(X_train, y_train)

    # Reporte de clasificación
    predictions = model.predict(X_test)
    print(classification_report(y_test, predictions, digits=4))
    
    accuracy = accuracy_score(y_test, predictions)
	
    # Matriz de confusión
    cm = confusion_matrix(y_test, predictions, labels=model.classes_)
    disp = ConfusionMatrixDisplay(
        confusion_matrix=cm, display_labels=model.classes_)
    disp.plot()

    return model,accuracy

## CLASSIFIERS (NB)

### CLASSIFIER 1

Representación: Bolsa de palabras con un vocabulario de 500 tokens, eliminando stop
words(ingles).

In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=500, stop_words=stopwords.words('english'))

# Ahora le solicitamos utilizando nuestro conjunto de datos que construya el vocabulario y tambien transforme nuestro texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

# Creamos el modelo y sus metricas
create_model(texto_features, "naives")

### CLASSIFIER 2

Representación: Bolsa de palabras con un vocabulario de 1000 tokens, eliminando stop
words(ingles)

In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=1000, stop_words=stopwords.words('english'))

# Ahora le solicitamos utilizando nuestro conjunto de datos que construya el vocabulario y tambien transforme nuestro texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

# Creamos el modelo y sus metricas
create_model(texto_features, "naives")

### CLASSIFIER 3

Representación: Bolsa de palabras con un vocabulario de 5000 tokens, eliminando stop
words(ingles).

In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=5000, stop_words=stopwords.words('english'))

# Ahora le solicitamos utilizando nuestro conjunto de datos que construya el vocabulario y tambien transforme nuestro texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

# Creamos el modelo y sus metricas
create_model(texto_features, "naives")

## CLASSFIER (SDG)

### CLASSIFIER 4


In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=5000, stop_words=stopwords.words('english'))

# Ahora le solicitamos utilizando nuestro conjunto de datos que construya el vocabulario y tambien transforme nuestro texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

# Creamos el modelo y sus metricas
model_words,_ = create_model(texto_features, "sdg", eta=0.1)

### CLASSIFIER 5

In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=5000, stop_words=stopwords.words('english'))

# Ahora le solicitamos utilizando nuestro conjunto de datos que construya el vocabulario y tambien transforme nuestro texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

# Creamos el modelo y sus metricas
create_model(texto_features, "sdg", eta=100)

# RESPUESTAS

## ¿Qué efecto sobre el F1-score y el accuracy tiene el incremento del vocabulario?, es bueno o negativo incrementarlo?

rpta:  `El incremento del vocabulario tuvo cambios positivos en cuanto al porcentaje de f1-score y el accuracy incrementandolo de 89. a 93., sin embargo, hubieron problemas relacionados con la ram al momento de generar la matriz de features para los vocabularios de 5000 palabras, esto se resolvio cambiando el tipo dato para las matrices de float64 a float16`

## ¿En base a los resultados del clasificador SGDClassifier y experimentación adicional que realice, para este problema que valor de la tasa de aprendizaje es apropiado?, ¿Vale la pena incrementarlo como en el clasificador 5?


Hallando la tasa de aprendizaje apropiado

In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=5000, stop_words=stopwords.words('english'))

# Ahora le solicitamos utilizando nuestro conjunto de datos que construya el vocabulario y tambien transforme nuestro texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

In [ ]:
tasa_acc = None
eta = 0.1
accuracy = 0
for i in range(10):
    _,acc = create_model(texto_features, "sdg", eta=eta)
    if acc >= accuracy:
        accuracy = acc
        tasa_acc = eta
    eta += 0.1

In [ ]:
print(f"{tasa_acc} - {accuracy}")

rpta: `Luego de realizar las pruebas cambiando e incrementando la tasa de aprendizaje, obtuve que el mejor resultado es asignandole un valor de 0.1, ya que al incrementar esa tasa el accuracy como el f1-score van disminuyendo`

## Los coeficientes que se obtienen del SGD son un indicativo de importancia de características. ¿Utilizando el clasificador 4, cuáles son las palabras más relevantes (importantes) para la tarea de clasificación?

In [ ]:
def top_words(palabras, coefs):
    words = []
    for i in range(len(coefs)):
        words.append((palabras[i],coefs[i]))

    positives = sorted(words, key=lambda x: (x[1]),reverse = True)
    negatives = sorted(words, key=lambda x: (x[1]),reverse = False)
    return positives,negatives
positives,negatives = top_words(vectorizer.get_feature_names_out(),model_words.coef_[0])
top_words = positives[:5] + negatives[:5]

In [ ]:
top_words = sorted(top_words, key=lambda x: ( abs(x[1])),reverse = True)
top_words


rpta: `Tras crear un touple donde guardamos la palabra con su coeficiente (peso|W), le realizamos los ordenamientos en base a su coeficiente y podemos observar las palabras mas importantes tanto negativas como positivas son las siguientes :`

`
	[('rotten'),
	('lame'),
	('letharg'),
	('vain'),
	('lousi'),
	('innoc'),
	('superior'),
	('intellig'),
	('divin'),
	('belov')]
`